# Define classes and consts

In [7]:
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from pathlib import Path
from typing import List, Dict
from os import listdir
from os.path import isfile, join
import re

In [8]:
LEAR_DATA_PATH = 'C:\my\study\IIAI\LB5\dataset_root'
TEST_DATA_PATH = 'C:\my\study\IIAI\LB5\input'
MODEL_INSTANCE_PATH = 'letter_recognition_cnn.pth'

class LetterRecognitionModel(nn.Module):
    def __init__(self):
        super(LetterRecognitionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 33)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class ImageToModelFormatConverter:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
        ])
    
    def process_single_image(self, path: str):
        image = Image.open(path)
        return self.transform(image).unsqueeze(0)  # Add batch dimension 


    def process_folder(self, folder_path: str):
        return ImageFolder(root=folder_path, transform=self.transform)


class FromTestDataSet:
    def __init__(self):
        self.converter = ImageToModelFormatConverter()
        self.pattern = pattern = re.compile(r'(.+)\.(jpg|png)$', re.IGNORECASE)
    
    def form_dataset(self, path: str) -> Dict['str', List]:
        data = {}
        for file in listdir(path):
            if (match := self.pattern.match(file)):
                name = match.group(1)
                print(name)
                if name not in data:
                    data[name] = []
                
                data[name] = self.converter.process_single_image(join(path, file))         

        return data

# Model learn

In [17]:
converter = ImageToModelFormatConverter()

dataloader = DataLoader(converter.process_folder(LEAR_DATA_PATH), batch_size=32, shuffle=True)

model = LetterRecognitionModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Save the trained model
torch.save(model.state_dict(), MODEL_INSTANCE_PATH)


Epoch 1/1000, Loss: 4.10499382019043
Epoch 2/1000, Loss: 3.5534284114837646
Epoch 3/1000, Loss: 3.4879136085510254
Epoch 4/1000, Loss: 3.462169647216797
Epoch 5/1000, Loss: 3.4276156425476074
Epoch 6/1000, Loss: 3.5784223079681396
Epoch 7/1000, Loss: 3.502145767211914
Epoch 8/1000, Loss: 3.4599266052246094
Epoch 9/1000, Loss: 3.4699959754943848
Epoch 10/1000, Loss: 3.4620585441589355
Epoch 11/1000, Loss: 3.4801599979400635
Epoch 12/1000, Loss: 3.4597201347351074
Epoch 13/1000, Loss: 3.528832197189331
Epoch 14/1000, Loss: 3.5097317695617676
Epoch 15/1000, Loss: 3.568312883377075
Epoch 16/1000, Loss: 3.527637004852295
Epoch 17/1000, Loss: 3.4504141807556152
Epoch 18/1000, Loss: 3.5019047260284424
Epoch 19/1000, Loss: 3.416905403137207
Epoch 20/1000, Loss: 3.5654244422912598
Epoch 21/1000, Loss: 3.448198080062866
Epoch 22/1000, Loss: 3.36281681060791
Epoch 23/1000, Loss: 3.5490520000457764
Epoch 24/1000, Loss: 3.4507641792297363
Epoch 25/1000, Loss: 3.335355043411255
Epoch 26/1000, Loss: 

# Model testing

In [14]:
if 'model' not in locals():
    model = LetterRecognitionModel()


model.load_state_dict(torch.load(MODEL_INSTANCE_PATH))
model.eval()

tests_loader = FromTestDataSet()

for letter, data in tests_loader.form_dataset(TEST_DATA_PATH).items():
    for test in data:

        with torch.no_grad():
            output = model(test)


        predicted_class = torch.argmax(output).item()
        predicted_letter = chr(predicted_class + ord('А'))

        print(f"Actual: {letter} Predicted: {predicted_letter}")


А
Б
В
З
И
Х
Щ
Ь
Э
Actual: А Predicted: Е
Actual: Б Predicted: И
Actual: В Predicted: Г
Actual: З Predicted: Ф
Actual: И Predicted: О
Actual: Х Predicted: А
Actual: Щ Predicted: Ъ
Actual: Ь Predicted: Э
Actual: Э Predicted: П
